In [5]:
from osgeo import gdal,ogr,osr

# Input raster file
raster_file = r'F:\wenqu\Aviris\site2a\PLSR\aviris_single_pixel\9_23.tif'

# Output polygon file
polygon_file = 'F:/wenqu/trait_map/output.shp'

In [6]:
# Open the input raster file
raster_ds = gdal.Open(raster_file)

# Get the raster band and its data type
band = raster_ds.GetRasterBand(1)
data_type = band.DataType

# Get the raster's geotransform and projection
geotransform = raster_ds.GetGeoTransform()
projection = raster_ds.GetProjection()


In [7]:
# Create a memory layer to hold the polygon features
driver = ogr.GetDriverByName("Memory")
mem_ds = driver.CreateDataSource("")

# Create a new layer in the memory dataset
mem_layer = mem_ds.CreateLayer("polygon", srs=osr.SpatialReference(wkt=projection), geom_type=ogr.wkbPolygon)

# Use gdal.Polygonize() to convert the raster to polygons
gdal.Polygonize(band, None, mem_layer, 0, [], callback=None)


0

In [8]:
print(mem_layer)

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x00000243B4D28B70> >


In [8]:
# Create the output shapefile
driver = ogr.GetDriverByName("ESRI Shapefile")
out_ds = driver.CreateDataSource(polygon_file)

# Copy the memory layer to the output shapefile
out_layer = out_ds.CopyLayer(mem_layer, "polygon")


In [9]:
# Clean up the memory layer
mem_ds.Destroy()

# Close the input and output files
raster_ds = None
out_ds = None


In [3]:
from osgeo import gdal, ogr

# Path to the input raster file
input_raster_path = 'F:/wenqu/trait_map/site2a_la_trait_map_project3.tif'

# Path to the shapefile used for clipping
shapefile_path = 'F:/wenqu/trait_map/output.shp'

# Path to the output clipped raster file
output_raster_path = 'F:/wenqu/trait_map/output_clipped_raster2.tif'

# Open the raster file
raster_dataset = gdal.Open(input_raster_path)

# Open the shapefile
shapefile_dataset = ogr.Open(shapefile_path)
shapefile_layer = shapefile_dataset.GetLayer()

# Clip the raster using the shapefile
outputBounds = shapefile_layer.GetExtent()
clipped_raster = gdal.Warp(output_raster_path, raster_dataset, cutlineLayer=shapefile_layer, cropToCutline=True, dstNodata=0, outputBounds=outputBounds)


# Close the datasets
raster_dataset = None
shapefile_dataset = None
clipped_raster = None

In [13]:
# Open the vector layer containing the polygons
vector_ds = ogr.Open('F:/wenqu/trait_map/output.shp')
vector_layer = vector_ds.GetLayer()

# Get the projection and geotransform information
proj = osr.SpatialReference(wkt=vector_layer.GetSpatialRef().ExportToWkt())

# Get the extent of the shapefile
xmin, xmax, ymin, ymax = vector_layer.GetExtent()

# Get the pixel size of the output raster
pixel_size = 30 # Replace with the actual pixel size of the output raster

# Calculate the number of pixels and lines in the output raster
xsize = int((xmax - xmin) / pixel_size)
ysize = int((ymax - ymin) / pixel_size)

# Get the geotransform of the output raster
gt = gdal.ApplyGeoTransform(vector_layer.GetSpatialRef().GetGeoTransform(), [xmin, pixel_size, 0, ymax, 0, -pixel_size])

# Open the raster file to be clipped
raster_ds = gdal.Open(r'F:\wenqu\trait_map\site2a_la_trait_map_project3.tif')

AttributeError: 'SpatialReference' object has no attribute 'GetGeoTransform'

In [14]:
# Create a new raster file to store the clipped result
driver = gdal.GetDriverByName('GTiff')
clipped_ds = driver.Create(r'F:\wenqu\trait_map\clipped_output.tif', xsize=raster_ds.RasterXSize, ysize=raster_ds.RasterYSize, bands=1, eType=raster_ds.GetRasterBand(1).DataType)
clipped_ds.SetProjection(proj.ExportToWkt())
clipped_ds.SetGeoTransform(gt)

# Clip the raster using the vector layer
gdal.RasterizeLayer(clipped_ds, [1], vector_layer, options=['ATTRIBUTE=pixel_value'])

# Close the raster and vector files
raster_ds = None
clipped_ds = None
vector_ds = None


TypeError: sequence must have length ##size